# Project Milestone 2

Here we will describe the whole pipeline to get all the results we would like to include in the final story (on the final website). We will go through all the different steps and describe in detail the operations needed. 

For the final story we decided to focus on the influence of the Brexit. More precisely we would like to assess how the Brexit was perceived and how it evolved over the years. The visualizations we aim at providing in the final story are detailed in this [Section](#Results).

## **[Preprocessing steps](#Preprocessing)**

As usual the first step consists in several substeps that aims at cleaning and transforming the data. By clicking on the task link, you can access the respective pipeline.
- *[Data exploration and Sanity check](#Sanity_check)* : Explore the dataset, check its consistency and get familiar with the different features/information provided.
- *[Data extraction](#extraction)* : Extract the data of interest that will be used to perform the tasks related to each idea.
- *[Data augmentation](#augmentation)* : Perform a data augmentation to get more features about the quotations such as the quote field, the nationality of the speaker and so on... These new features will be used to perform the tasks related to each idea.
- *[Data cleaning and merging](#augmentation)* : Perform a final cleaning on the quotations as well as on the speakers and generate 3 main datasets that will be used for the analysis
- *[Quotations and speakers clustering](#clustering)* : Cluster the quotations and the speakers according to the quotation vector and the added features in the data augmentation task. This clustering will be mainly used to develop a recommandation tool.

## **[Generate the results for the final story](#Results)**

- [General Statitics](#Statistics) : Explore the dataset, visualize some first graphs for each new features.  
- [Country map](#Country) : Show how brexit is perceived depending on the country.
- [Sector map](#Sector) : Show how brexit is perceived depending on the sector.
- [Visualize speakers evolution](#2Dplot) : Visualize speakers into an embedding space that should reflect the similarities between speakers **[TO BE COMPLETED]**.
- [Recommandation Tool](#Recommandation) : Tool that recommends similar speakers to the one searched by the user **[TO BE COMPLETED]**. 
- [Correlation with stocks](#Stocks) : Study if a correlation exists between remarkable Brexit peaks and the stock actions from companies of the FTSE100 **[TO BE COMPLETED]**.


# Before diving into the code 

To run everything from scratch, make sure to have a `Data` folder containing the following files: 
- The quotebank datasets for each year: `quotes-yyyy.json.bz2`
- The speaker attributes folder `speaker-attributes.parquet` as well as the associated lookup table `wikidata_labels_descriptions_quotebank.csv.bz2`

To benefit from check points, download `Brexit_datas` from [Google drive](https://drive.google.com/drive/folders/12EgO7E97KcNrZtQhjUmkOp5iDF1V7ufR?usp=sharing)


## Import useful librairies and define useful variables

In [ ]:
# STANDARD LIBRAIRIES
from os.path import exists
import os
import bz2 
import json
import geojson
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import itertools
from datetime import datetime
from scipy import stats 

# Dynamic graphs
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# Machine learning librairies
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import SpectralClustering, DBSCAN, AgglomerativeClustering
from sklearn import metrics
#from sklearn.manifold import TSNE
from tsne_torch import TorchTSNE as TSNE
from sklearn.manifold import LocallyLinearEmbedding
import nltk
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Cross validation
from sklearn.model_selection import cross_val_score

# Graph algorithms
import networkx as nx

# import string distances
import stringdist

# Load the lexicon for sentiment analysis
nltk.download('vader_lexicon')

import warnings
warnings.filterwarnings("ignore")

# Data files
PATHS_TO_FILE = ['Data/quotes-20%d.json.bz2' % i for i in range(15,21)]

# Columns to analyte for one-hot vectorization task
columns_to_map = ['nationality', 'gender', 'occupation', 'party', 'academic_degree', 'religion']

# time discretization by periods
period_labels = ["< 2018", "2018", "early 2019", "late 2019 - 2020"]

# type of the sentence_transformer
sentence_transformer_type = 'all-MiniLM-L6-v2'

<a id='Preprocessing'></a>

# Preprocessing steps

<a id='Sanity_check'></a>

## Data exploration and Sanity check

We decided to perform the following snaity checks on the original data: 

- We first check that each entry for each quotation is specified in the right format (e.g. `numOccurences` should be an integer).
- We check that the `probas` sum to 1.
- We check that the `numOccurences` is superior or equal to the length of the list containing the urls.
- The `date` is consistent with the dataset they are coming from
- We check that if a `qids` exists then a `speaker` should be specified

In [ ]:
# SANITY CHECK FUNCTIONS

def check_type(instance,entry,dtype):
    return type(instance[entry]) == dtype

def check_probas(instance):
    if len(instance) > 0:
        proba_sum = sum([float(potential[1]) for potential in instance["probas"]])
        if proba_sum < 0.98 or proba_sum > 1.02:
            return False
        else:
            return True
    else:
        return False

def check_numOcc(instance):
    return (len(instance["urls"]) <= instance["numOccurrences"])

def check_date(instance,year):
    quotation_year = int(instance["date"][:4])
    return (quotation_year == year)

def check_author_qids(instance):
    if len(instance["qids"]) > 0 and instance["speaker"] is None:
        return False
    else: 
        return True
        
# CONVERSION FUNCTIONS

# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

# ANALYSIS AND STANDARDIZATION

def sentiment_to_int(value):
  return int(value == "Positive") - int(value == "Negative")

def standardize(df):
    return (df - df.mean(axis=0)) / df.std(axis=0)

In [ ]:
# Define the types for each entry
TYPES = {"quoteID":str,
         "quotation":str,
         "speaker":str,
         "qids":list,
         "date":str,
         "numOccurrences":int,
         "probas":list,
         "urls":list,
         "phase":str}

error_file = "Data/error_file.json.bz2"


if not exists(error_file):
    with bz2.open(error_file, 'wb') as e_file:
        # Loop over the different files that we will read
        for quotebank_data in PATHS_TO_FILE:
            year = int(quotebank_data[-13:-9])
            print("Reading ",quotebank_data," file...")
            # Open the file we want to read
            with bz2.open(quotebank_data, 'rb') as s_file:
                # Loop over the samples
                for instance in s_file:
                    potential_error = ""
                    # Loading a sample
                    instance = json.loads(instance)
                    #### CHECK THE TYPES ####
                    for key, value in TYPES.items():
                        if not check_type(instance,key,value):
                            potential_error += "| Type problem: " + key + " |"
                            # Continue because there exists a problem with the type that may affect the other checks
                            continue
                    #### CHECK THE PROBAS ####
                    if not check_probas(instance):
                        potential_error += "| Probas problem |"
                    #### CHECK THE DATE ####
                    if not check_date(instance,year):
                        potential_error += "| Date problem |"
                    #### CHECK THE NUMOCCURENCES ####
                    if not check_numOcc(instance):
                        potential_error += "| NumOccurences problem |"
                    #### CHECK THE AUTHOR-QIDS ####
                    if not check_author_qids(instance):
                        potential_error += "| Author-qids problem |"
                    # WRITE INTO THE FILE FOR POTENTIAL ERRORS #
                    if len(potential_error) > 0:
                        instance["error"] = potential_error
                        e_file.write((json.dumps(instance)+'\n').encode('utf-8'))

pd.read_json('Data/error_file.json.bz2',compression="bz2",lines=True).shape                    

<a id='extraction'></a>

## Data extraction

As mentionned previously, we are planning to analyze the way Brexit is perceived. Thus, we need to extract first the quotations that discuss Brexit. To do so we will follow the following pipeline:

1. Select all the quotations that contain the word Brexit.
2. Store the new two dataset in the `Brexit_quotes.json.bz2` file.


In [ ]:
if not exists('Brexit_datas/Brexit_quotes.json.bz2'):
    # Input file
    PATHS_TO_FILE = ['Data/quotes-20%d.json.bz2' % i for i in range(15,21)]
    # Output file
    PATH_TO_OUT = 'Brexit_datas/Brexit_quotes.json.bz2'

    # Open the file where we will write
    with bz2.open(PATH_TO_OUT, 'wb') as d_file:
        # Loop over the different files that we will read
        for quotebank_data in PATHS_TO_FILE:
            print("Reading ",quotebank_data," file...")
            # Open the file we want to read
            with bz2.open(quotebank_data, 'rb') as s_file:
                # Loop over the samples
                for instance in s_file:
                    # Loading a sample
                    instance = json.loads(instance)
                    # Extracting the quotation
                    quotation = instance['quotation']
                    # Check if the quotation contains at least one word related to Brexit
                    if "brexit" in quotation.lower():
                        # Writing in the new file
                        d_file.write((json.dumps(instance)+'\n').encode('utf-8'))

quotebank_brexit = pd.read_json('Brexit_datas/Brexit_quotes.json.bz2',compression="bz2",lines=True)
quotebank_brexit.head(2)


<a id='augmentation'></a>

## Data augmentation

When we will generate the results for the final story, we will need more information than the initial features provided. The further analysis will require to have access to other features such as the sentiment carried by the quotation and additional information about the author. To do so, the following pipeline will be performed on each quotation:

1. **[Adding features related to the author](#Features_Author)** :  Using the provided file `speaker_attributes.parquet` that was extracted from the Wikidata knowledge base, the following attributes are of interest for each speaker:
    - `occupation`: describes the author's occupancy 
    - `party` identifies the political affiliation of the speaker.
    - `academic_degree` gives information about the education of the author as well as their alma mater.
    - `nationality` identifies the citizenship(s) of the author.
    - `date_of_birth`: identifies the date of birth of the speaker.
    - `gender`: identifies the gender of the speaker.
    - `ethnic_group`: identifies the ethnic group of the speaker.
    - `religion`: identifies the religion of the speaker. 

    The provided `speaker_attributes.parquet` file contains attributes in terms of QIDs, thereby being uninterpretable by humans. To map the QIDs to meaningful labels, we used the provide the file `wikidata_labels_descriptions_quotebank.csv.bz`.
    
    The aforementioned attributes may not be available for all authors. When it is the case, a NaN value is assigned.

2. **[Adding features issued from a sentiment analysis](#Sentiment_Quote)** : The last feature of interest is the sentiment that is carried by the quotation. For the sake of simplicity, each quotation will be classified into three different categories: *Negative*, *Neutral* and *Positive*. 
Sentiment Analysis task can be performed using pretrained Deep Neural Networks. We decided to use **Vader** Neural network for its good performance. NLTK's Vader sentiment analysis tool uses a bag of words approach with some simple heuristics. More on it [here](https://github.com/cjhutto/vaderSentiment). 

<a id='Features_Author'></a>
***Loading the speaker_attributes.parquet file***

In [ ]:
# Load the parquet that contains the information about speakers
df_attributes = pd.read_parquet('Data/speaker_attributes.parquet')

# we are not interested in the aliases, lastrevid, US_congress_bio_ID, id, candidacy and type.
keep_attributes = ['id','label', 'date_of_birth', 'nationality', 'gender', 'ethnic_group', 'occupation', 'party', 'academic_degree', 'religion']
# Set the index
df_attributes = df_attributes[keep_attributes].set_index('id')
# Sanity check for the qids
print("Sanity check ok ? : ",df_attributes.index.is_unique)
# Let's have a look
df_attributes.sample(2)

<a id='Features_Author'></a>
***Mapping the QIDs to meaningful labels***

In [ ]:
# create dictionnary to use it as a lookup table 
df_map = pd.read_csv('Data/wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')
# Dictionnary where qids are keys and values are corresponding element
map_dict = df_map.Label.to_dict()

def mapping(QIDs):
    """
    The purpose of this function is to map all the QIDs to their labels, 
    using wikidata_labels_descriptions_quotebank.csv
    """
    
    if QIDs is None:
        return np.nan
    else:
        QIDs_mapped = []
        for QID in QIDs:
            try:
                # If a correspondance exists
                QIDs_mapped.append(map_dict[QID])
            except KeyError:
                # If no correspondance exits
                continue
        # If nothing was extracted
        if len(QIDs_mapped) == 0:
            return np.nan
        # Things extracted
        else:
            return QIDs_mapped



# For each column perform the mapping to transform qids to real value
for column in columns_to_map:
    df_attributes[column] = df_attributes[column].apply(mapping)
    
df_attributes.head(2)

<a id='Sentiment_Quote'></a>
***Adding sentiment score to each quote***

In [ ]:
def sent_score(quote):
    """The purpose of this function is to use the sentiment analysis tool VADER to find the sentiment associated with a quote."""
    
    sid = SentimentIntensityAnalyzer()
    sentiment_dict = sid.polarity_scores(quote)
    
    # The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between
    # -1(most extreme negative) and +1 (most extreme positive).
    # positive sentiment : (compound score >= 0.05) 
    # neutral sentiment : (compound score > -0.05) and (compound score < 0.05) 
    # negative sentiment : (compound score <= -0.05)
    # see https://predictivehacks.com/how-to-run-sentiment-analysis-in-python-using-vader/
    # or https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/
    
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return "Negative" 
 
    else :
        return "Neutral"

# backup quotebank dataframe with sentiment score if the corresponding file doesn't exists
if not exists("Brexit_datas/quotebank_brexit_with_sentiment.json.bz2"):
    quotebank_brexit['sentiment_score'] = quotebank_brexit.quotation.apply(sent_score) 
    quotebank_brexit.to_json("Brexit_datas/quotebank_brexit_with_sentiment.json.bz2")
    
else:
    quotebank_brexit = pd.read_json("Brexit_datas/quotebank_brexit_with_sentiment.json.bz2",compression="bz2")

quotebank_brexit.head(2)

<a id='cleaning'></a>

## Data merging and cleaning

Depending on the different tasks we want to perform we will need to have the dataset in various forms, thus we will generate three types of dataset: 
- `quotebank_brexit`: original dataset [+sentiment score] where dublicated quotations are removed.
- `aug_quotebank_brexit`: dataset with augmented data where both quotations and speakers are cleaned.
- `oneh_quotebank_brexit`: copy of `aug_quotebank_brexit` where categorical values are one-hot encoded.

Thus we first start by [cleaning the quotations](#cleaning_quotation) in the `quotebank_brexit` dataset and then we [clean the speakers](#cleaning_speaker) to be able to merge with augmented data and generate the `aug_quotebank_brexit`. After a [processing](#aug_preprocessing) of the `aug_quotebank_brexit` we will finally [one hot encode](#one_hot_encoding) to generate the `oneh_quotebank_brexit` dataset.

<a id='cleaning_quotation'></a>

### Cleaning the quotations

We noticed that some quotations were very similar, actually too similar. They sometimes differ from the fact that one quotation is nested in another or sometimes they only differ by one character. Here is an example of such a quotation:
- quoteID: **2018-01-26-042810** - *"I look at Nigel Farage's example. It took 17 years, but Brexit came,"*
- quoteID: **2018-01-26-042811** - *"I look at Nigel Farage's example. It took 17 years, but Brexit came. I don't plan to wait that long"*

We need to remove these kind of *duplicates*. To do so we followed this pipeline:
- Converting quotations into vectors using [SentenceTransformer](https://www.sbert.net/docs/usage/semantic_textual_similarity.html) deep neural network.
- Computing [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) between each pair of quotations
- Removing quotations that are too similar from the dataset

In [ ]:
# Encode quotation 
if not exists("Brexit_datas/vector_quotes.csv.gz"):
    encoder = SentenceTransformer(sentence_transformer_type)
    # Encode quotations
    quotes_encoded = encoder.encode(quotebank_brexit['quotation'].values, convert_to_numpy=True, show_progress_bar=True)
    # Convert to df
    quotes_df = pd.DataFrame(quotes_encoded, index = quotebank_brexit.index)
    # Add significant columns
    quotes_df["speaker"] = quotebank_brexit["speaker"].values
    # Export into a compressed format
    quotes_df.to_csv("Brexit_datas/vector_quotes.csv.gz")
    
else:
    # Read the file
    quotes_df = pd.read_csv("Brexit_datas/vector_quotes.csv.gz",index_col=0,compression="gzip")

quotes_df.head()

In [ ]:


# compare pairwise similarity
def filter_similar(df):
    # Get the embeddings computed before
    embeddings = df_to_tensor(df.drop(columns='quoteID'))
    # Compute cosine similarity
    cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
    # Convert to df
    score = pd.DataFrame(cosine_scores.numpy())
    index = set(score[score>0.95].stack().index.tolist())
    index = [(a,b) for (a,b) in index if a != b]
    # multiple tuples can have common element: need to merge them
    graph = nx.Graph(index)
    index = list(nx.connected_components(graph))
    # map the indices with the Qid of the quote
    index = [tuple(df.quoteID.iloc[ind] for ind in path) for path in index]
    return index

similar_count = quotes_df.assign(quoteID=quotebank_brexit.quoteID.values).groupby('speaker').apply(filter_similar)

In [ ]:
# an example of two similar quotations
print("Yvette Cooper:", similar_count["Yvette Cooper"][0])
quotebank_brexit[(quotebank_brexit.quoteID=='2019-01-27-041304') | (quotebank_brexit.quoteID=='2019-01-27-029003')]

In [ ]:
# get indices to drop
def drop_duplicate_quotes(ids):
    return [[quoteID for quoteID in path[1:]] for path in ids]
        
# generate the list of quoteIDs to be removed
to_be_removed = similar_count.apply(drop_duplicate_quotes).values.sum()
to_be_removed = list(itertools.chain.from_iterable(to_be_removed))

quotebank_brexit = quotebank_brexit[~quotebank_brexit.quoteID.isin(to_be_removed)]

print("Number of quotations to be removed: ",len(to_be_removed))

In [ ]:
file_quote_bank = "Brexit_datas/quotebank_brexit_cleaned.json.bz2"

if not exists(file_quote_bank):
    quotebank_brexit.to_json(file_quote_bank)
else:
    quotebank_brexit = pd.read_json(file_quote_bank,compression="bz2")

<a id='cleaning_speaker'></a>

### Cleaning the speakers

The `aug_quotebank_brexit` provides information about the speaker such as the `nationality`, `occupation`. However one can notice that sometimes the neural network doesn't succeed in finding a speaker and therefore fills `speaker` entry with `None` value. These missing values are difficult to handle as it would require to guess who said the quotation. One could think about training a classifier on the data where the speaker is mentionned but it is actually a fastidious task that we are not able to manage. Unfortunaltely, we decided to remove them from the dataset.

An other issue comes from the fact that for one speaker different Qids exist. However, these Qids correspond to the Wikipedia pages of the same person but in different langagues. This could also come from the fact that there exist multiple wikipedia pages that point to different persons who are homonyms. When many qids exist we check if all the attributes are similar for all the qids. If not, then we are not able to determine which qid is the correct one so unfortunately we discard the row from the dataset.

In [ ]:
def check_consistent_qids(QIDS_original):
    QIDS = QIDS_original.copy()
    if len(QIDS) == 0:
        return pd.NA
    elif len(QIDS) == 1:
        return QIDS_original[0]
    else:
        while len(QIDS) > 1:
            first_idx = QIDS.pop(-1)
            try:
                first = df_attributes.loc[first_idx].fillna(0)
                second_idx = QIDS.pop(-1)
                try:
                    second = df_attributes.loc[second_idx].fillna(0)
                except KeyError:
                    QIDS.append(first_idx)
                    continue
            except KeyError:
                continue
            try: 
                if (first != second).sum() > 0:
                    return pd.NA
            except ValueError:
                return pd.NA
        return QIDS_original[0]

if not exists("Brexit_datas/aug_quotebank.json.bz2"):
    # Remove nan values
    aug_quotebank_brexit = quotebank_brexit[quotebank_brexit.speaker != "None"]

    # Remove speakers with multiple different qids
    aug_quotebank_brexit.qids = aug_quotebank_brexit.qids.apply(check_consistent_qids)
    aug_quotebank_brexit = aug_quotebank_brexit[~aug_quotebank_brexit.qids.isna()]

    # Merge the augmented quotebank brexit with df_attributes on qids
    aug_quotebank_brexit = pd.merge(aug_quotebank_brexit, df_attributes, 'inner', left_on="qids", right_index=True)

    # Export to json to add check points
    aug_quotebank_brexit.to_json("Brexit_datas/aug_quotebank.json.bz2")
else:
    # Read json if it already exists
    aug_quotebank_brexit = pd.read_json("Brexit_datas/aug_quotebank.json.bz2",compression="bz2")

# Let's have a look
print("New shape:",aug_quotebank_brexit.shape)
aug_quotebank_brexit.head(2)

### Compute age feature for each speaker

In [ ]:
def get_age(birth_date,current_year=datetime.now().year):
    
    # Check it is a list 
    if isinstance(birth_date,list) and len(birth_date) > 0:
        # Only get the first 4 digit (year)
        birth_year = int(birth_date[0][1:5])
        # Return the age
        return current_year - birth_year
    else:
        return pd.NA

# Add the age column
aug_quotebank_brexit["Age"] = aug_quotebank_brexit.date_of_birth.apply(get_age)

aug_quotebank_brexit.loc[:,["Age","date_of_birth","speaker"]].sample(5)

### Get unique values of categorical features

In [ ]:
unique_values = {}

# For each categotical value
for col in columns_to_map:
    # Get the serie
    col_serie = aug_quotebank_brexit[col].copy()
    # Get unique values
    unique_values[col] = pd.unique(col_serie.apply(pd.Series).stack())
    print(col," : number of different categories = ",len(unique_values[col]))

<a id='aug_preprocessing'></a>

### What about the categorical features added

### Occupation feature

Now that we added new features, we had a look at their values. We noticed that there are more than 800 different occupations. It would be interesting to classify them into *categories*. The problem is that we do not have any label on them and using ML techniques such as pre-trained neural networks would be an over-kill. We rather followed a semi-manual approach that is described below: 
- Identify which words are the more frequent in the `occupation` names and associate them with a label. We will call them key words.
- For each `occupation` match it with any key word labels when applicable.
- Label the remaining occupations manually.

***Manage frequent keywords***

In [ ]:
# Data frame of the occupations
occupation_df = pd.DataFrame(unique_values["occupation"],columns=["occupation"])

key_words = []

# Loop over the occupations
for occupation in unique_values["occupation"]:
    # Split the occupation string and concatenate
    key_words += occupation.split()

# Convert to a Dataframe
key_words_df = pd.DataFrame(key_words,columns=["occupation"])
# Put all strings to lower
key_words_df.occupation = key_words_df.occupation.str.lower()
# For each key word count the number of occurences and sort by descending
key_words_df = key_words_df.groupby("occupation").size().reset_index(name="Count").sort_values(by="Count",ascending=False)

# If the classification has not been already done
if not exists("Brexit_datas/occupation_class/occupation_agg.csv"):
    key_words_df.to_csv("Brexit_datas/occupation_class/occupation_agg.csv")

print("Look at the most frequent keywords")
print(key_words_df.head(3))

answer = input("Is the classification of keywords done ?")

if (answer.lower() == "yes"):
    # Get the classified keywords
    key_words_classified = pd.read_csv("Brexit_datas/occupation_class/occupation_agg.csv",index_col=0)
    # Get ride of keywords that have not been classified
    key_words_classified = key_words_classified.loc[~key_words_classified.Category.isna()]
    # Manage the case when several categories have been entered
    key_words_classified.Category = key_words_classified.Category.apply(lambda x: x.split("-"))
    # let's have a look at the table
    print("Look at the output table")
    print(key_words_classified.head(3))
else:
    print("Then please classify the keywords")

***Match occupation and keyword labels***

In [ ]:
# Function to check if keywords are contained in an occupation
def check_string_in(occupation):
    # Initialize the final list of the supercategories
    final_list = []
    # Loop over the key_words_classified
    for items in key_words_classified.occupation.iteritems():
        # If the keyword is contained in the occupation
        if items[1] in occupation.lower():
            # Concat the supercategories with th existing list
            final_list = final_list + key_words_classified.loc[items[0],"Category"]
    # If no categories return NaN
    if len(final_list) == 0:
        return pd.NA
    # Else return the list without duplicates
    else:
        return list(set(final_list))
        
# Apply the function
occupation_df["Category"] = occupation_df.occupation.apply(check_string_in)

if not exists("Brexit_datas/occupation_class/unclassified_occupation.csv"):
    # Export the occupations that have not been classified
    occupation_df[occupation_df.Category.isna()].to_csv("Brexit_datas/occupation_class/unclassified_occupation.csv")

print("Look at the remaining occupations")
print(occupation_df[occupation_df.Category.isna()].head(3))

answer = input("Is the classification of remaining occupations done ?")

if (answer.lower() == "yes"):
    # Get the remaining occupations classified
    remain_occupations_classified = pd.read_csv("Brexit_datas/occupation_class/unclassified_occupation.csv",index_col=0)
    # Merge with the current data frame
    occupation_final_df = pd.merge(occupation_df,remain_occupations_classified,how="left",on="occupation",suffixes=("","_2"))
    # Split into a list
    occupation_final_df.Category_2 = occupation_final_df.Category_2.apply(lambda x: x.split("-") if type(x) == str else pd.NA)
    # Merge into a single column
    occupation_final_df.loc[~occupation_final_df.Category_2.isna(),"Category"] = occupation_final_df.loc[~occupation_final_df.Category_2.isna(),"Category_2"]
    # Drop the artificial column
    occupation_final_df.drop(columns=["Category_2"],inplace=True)
    # Drop na values that corresponds to unclassifiable jobs such as nazi hunter
    occupation_final_df.dropna(axis=0,inplace=True)
    # Let's have a look
    print("Final data set for the classification of occupations:")
    print(occupation_final_df.head(5))
    # Export to a json file
    if not exists("Brexit_datas/occupation_class/classified_occupation.json"):
        occupation_final_df.to_json("Brexit_datas/occupation_class/classified_occupation.json")
else:
    print("Then please classify the remaining occupations")

***Label remaining occupations***

In [ ]:
occupation_final_df = pd.read_json("Brexit_datas/occupation_class/classified_occupation.json").set_index("occupation")

# Let's have a look at the supercategories
print(list(pd.unique(occupation_final_df.Category.apply(pd.Series).stack())))

# Let's replace this into the aug_quotebank dataset
def replace_occupation(occupation):
    if type(occupation) == list:
        if len(occupation) > 0:
            new_occupation = []
            for job in occupation:
                try:
                    new_occupation += occupation_final_df.loc[job,"Category"]
                except KeyError:
                    continue
            if len(new_occupation) > 0:
                return list(set(new_occupation))
            else:
                return pd.NA
                
    else:
        return pd.NA

aug_quotebank_brexit.occupation = aug_quotebank_brexit.occupation.apply(replace_occupation)
aug_quotebank_brexit.head(2)
        

### Country feature

In [ ]:
def compare_levensthein(country,proposal=5):

    # Given the initial country, should we deviate from this initial one ?
    message = country + " Any deviation ?"
    deviation = input(message)

    # If a deviation was specified then replace
    if len(deviation) > 0:
        country = deviation
    
    # Compute the levenshtein distance with the normalized country
    value = []
    for existing in list(current_countries.Country):
        leven_distance = stringdist.levenshtein_norm(country,existing)
        value.append(leven_distance)
        
    value = np.array(value)
    # Sort the countries according to their closeness
    closer = current_countries.Country.values[np.argsort(value)]
    # Get the "proposal" closest countries
    closer = closer[:proposal]

    # Display potential countries
    message = country + " potential candidates \n" + " --- ".join(list(closer))

    fine = True
    while fine:
        try:
            # Give the index of the potential country that will replace the initial country
            idx_to_keep = int(input(message))
            fine = False
        except ValueError:
            print("Please specify an integer")
            fine = True

    # If negative index then discard
    if idx_to_keep < 0:
        return pd.NA
    # Else return the closer one
    else:
        return closer[idx_to_keep]


if not exists("Brexit_datas/country_class/country_final_mapping.csv"):

    # Get the list of existing countries 
    current_countries = pd.read_excel("Brexit_datas/country_class/countries.xlsx")

    # Remove capital letters and special characters
    current_countries.Country = current_countries.Country.str.lower()
    current_countries.Country = current_countries.Country.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Country values that we currently have
    countries_to_map = pd.DataFrame(unique_values["nationality"],columns=["Country"])
    # Remove capital letters and special characters
    countries_to_map.Country = countries_to_map.Country.str.lower()
    countries_to_map.Country = countries_to_map.Country.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # Drop eventual duplicates
    countries_to_map.drop_duplicates(subset=["Country"],inplace=True)

    # Let's perform a first merge
    countries_to_map = pd.merge(current_countries,countries_to_map,left_on="Country",right_on="Country",how="right")

    # New column new countries
    countries_to_map.loc[~countries_to_map.ISO.isna(),["real_country"]] = countries_to_map[~countries_to_map.ISO.isna()].Country
    # Fill the remaining countries manually
    countries_to_map[countries_to_map.ISO.isna()].real_country= countries_to_map[countries_to_map.ISO.isna()].Country.apply(compare_levensthein)
    # Remove countries that didn't find a correspondance
    countries_to_map = countries_to_map[~countries_to_map.real_country.isna()].drop(columns="ISO")
    # Export the result
    countries_to_map.to_csv("Brexit_datas/country_class/country_final_mapping.csv",index_col=0)

else:
    # Read the already created csv
    countries_to_map = pd.read_csv("Brexit_datas/country_class/country_final_mapping.csv",index_col=0)

# Set the index 
countries_to_map.set_index("Country",inplace=True)
countries_to_map.head()

In [ ]:
def match_category(initial,lookup_table):

    # Initialize the potential new_values
    new_values = []

    # Get the original values
    original = list(lookup_table.index.values)
    # Check that the element is a list
    if isinstance(initial,list):
        # For each word in the list
        for old_original in initial:
            # Check if there exists a corresponding word
            if old_original.lower() in original:
                # Then get the new value
                new_values.append(lookup_table.loc[old_original.lower(),lookup_table.columns[0]])
        if len(new_values) == 0:
            return pd.NA
        else:
            return new_values
    else:
        return pd.NA

# Replace initial country by normalized countries
aug_quotebank_brexit.loc[:,"nationality"] = aug_quotebank_brexit.nationality.apply(match_category,lookup_table=countries_to_map)

## Thresholding to remove categories with low number of occurences

In [ ]:
threshold_maps = {}

# define threshold for each target column
threshold_maps["nationality"] = 80
threshold_maps["party"] = 50
threshold_maps["academic_degree"] = 20
threshold_maps["religion"] = 50
threshold_maps["gender"] = 50

# function filtering 
def intersection_test(xlist, unique_set):
    if not isinstance(xlist, list):
        return pd.NA
    if len(xlist) == 0:
        return pd.NA
    new_list = list(set(xlist).intersection(unique_set))
    if len(new_list) > 0:
        return new_list
    else:
        return pd.NA


for col, threshold in threshold_maps.items():
    print("Column", col, ", Keep only values with occurrence >", threshold)
    # get unique quantities inside list objects and their count
    unique_count = aug_quotebank_brexit[col].apply(pd.Series).stack().to_frame().rename(columns={0:"Value"})
    unique_count = unique_count.groupby("Value").size().reset_index(name="Count").sort_values(by="Count",ascending=False)
    # define intersection for the specific unique values
    intersection_unique = lambda xlist: intersection_test(xlist, unique_count[unique_count["Count"] > threshold].Value.values)
    # apply filtering
    aug_quotebank_brexit[col] = aug_quotebank_brexit[col].apply(intersection_unique)
    

## Academic degree gathering

We would like to gather academic degree of people into a higher categories such as PhD, professor, master bachelor and so on. As the number of different remaining categories for the academic degree is quite low, it can be done manually.

In [ ]:
# File where we will match higher categories
file_academic = "Brexit_datas/academic_degree/original_academic.csv"

if not exists(file_academic):
    # Get the unique values for academic
    Academic_filter = aug_quotebank_brexit["academic_degree"].apply(pd.Series).stack().to_frame().rename(columns={0:"Value"})
    # Group by value, count and export
    Academic_filter.groupby("Value").count().to_csv(file_academic)

# Please fill the file 
answer = input("Did you gather academic degrees categories into higher categories ?")

if answer.lower() == "yes":
    # Read the updated matching file 
    Academic_filter = pd.read_csv(file_academic).set_index("Value")
    # Remove capital letters
    Academic_filter.index = Academic_filter.index.str.lower()
    # match category
    aug_quotebank_brexit.loc[:,"academic_degree"] = aug_quotebank_brexit.academic_degree.apply(match_category,lookup_table=Academic_filter)

In [ ]:

# We want to remove useless information: for instance 
# someone with bachelor and Phd should be replaced by Phd only
def acadedmic_order(degrees,order):
    
    # Initialize max and argmax
    max = -1
    new = None

    if isinstance(degrees,list):
        # Loop over its potential degrees
        for degree in degrees:
            # Replace if higher degree 
            if max < order[degree]:
                max = order[degree]
                new = degree
        if new is None:
            print("Empty list")
            return pd.NA
        return [new]
    return pd.NA

# Define the order between degrees 
Ordered_list = {"Professor":4,"Phd":3,"Master":2,"Bachelor":1,"Other":0}

# Apply that man :)
aug_quotebank_brexit.loc[:,"academic_degree"] = aug_quotebank_brexit.academic_degree.apply(acadedmic_order,order=Ordered_list)

## Religion gathering

We would like to actually do the same thing for the religion feature

In [ ]:
file_religion = "Brexit_datas/religion/original_religion.csv"

if not exists(file_religion):
    Religion_filter = aug_quotebank_brexit["religion"].apply(pd.Series).stack().to_frame().rename(columns={0:"Value"})
    Religion_filter.groupby("Value").count().to_csv(file_religion)

answer = input("Did you gather religion categories into higher categories ?")

if answer.lower() == "yes":
    Religion_filter = pd.read_csv(file_religion).set_index("Value")
    Religion_filter.index = Religion_filter.index.str.lower()
    Religion_filter.head()
    aug_quotebank_brexit.loc[:,"religion"] = aug_quotebank_brexit.religion.apply(match_category,lookup_table=Religion_filter)

### Generate period for each quotation

In [ ]:
file_aug = "Brexit_datas/aug_quotebank_brexit_true.json.bz2"

if not exists(file_aug):
    aug_quotebank_brexit.to_json(file_aug)
else:
    aug_quotebank_brexit = pd.read_json(file_aug,compression="bz2")

In [ ]:
# define period cut (6 = 1 year, 12 = 6 months)
periods_nb = 12

# Visualize with an histogram on the date column
fig = px.histogram(aug_quotebank_brexit,x="date", nbins=12)#, color_discrete_sequence = px.colors.cyclical.Phase)
fig.update_layout(title="Number of quotations about Brexit accross time")
fig.show()

# Get period cut
period_list = list(range(periods_nb - 1))
period_cut = pd.date_range('2015-01-01', freq='6M', periods=periods_nb)

aug_quotebank_brexit["period"] = pd.cut(aug_quotebank_brexit.date.astype(np.int64)//10**9,
                   bins=period_cut.astype(np.int64)//10**9,
                   labels=period_list)

aug_quotebank_brexit.head(2)

Manually select and eventually merge periods in order to have almost a equal distribution between periods.

In [ ]:
# Manually merge periods (WARNING: it depends on periods )

def reset_periods(x):
    if x <= 5:
        return period_labels[0]
    elif x in [6, 7]:
        return period_labels[1]
    elif x == 8:
        return period_labels[2]
    else:
        return period_labels[3]
    
if (type(aug_quotebank_brexit["period"].dtypes) is pd.core.dtypes.dtypes.CategoricalDtype):
    # print counts
    print("Count occurrences for each periods, they must correspond to the histogram above.")
    for i in period_list:
        print("Size of period", i, ":", len(aug_quotebank_brexit.loc[aug_quotebank_brexit.period == i]))

    # apply period simplification
    aug_quotebank_brexit["period"] = aug_quotebank_brexit["period"].apply(reset_periods)


print("Check the occurrences after the period recalibration")
for i in period_labels:
    print("Size of period", i, ":", len(aug_quotebank_brexit.loc[aug_quotebank_brexit.period == i]))

<a id='one_hot_encoding'></a>

### One hot encoding

In [ ]:
unique_values = {}
# One hot vectorization of columns cotaining categorical values
dummy_col = "AAADummy column for the sake"
# Make a copy
oneh_quotebank_brexit = aug_quotebank_brexit.copy()

# Check that the element is a list that contains only one string
def ensure_list(value):
  if isinstance(value, list):
    for i in range(len(value)):
      value[i] = str(value[i])
  elif not pd.isna(value):
    value = [value]
  return value

# Loop over categorical columns
for col in columns_to_map:
  # Get the serie
  col_serie = aug_quotebank_brexit[col].copy().apply(ensure_list)
  # Change nan values to a list containing a dummy column
  col_serie[col_serie.isna()] = col_serie[col_serie.isna()].apply(lambda x: [dummy_col])
    
  # One hot vectorize
  categorical_df = pd.get_dummies(col_serie.apply(pd.Series).stack()).groupby(level=0).sum()
    
  # Drop the dummy column
  categorical_df.drop(columns=[dummy_col],inplace=True)
  # Refresh unique values
  unique_values[col] = categorical_df.columns
    
  # Join with quotebank brexit
  oneh_quotebank_brexit = oneh_quotebank_brexit.join(categorical_df,how="left",rsuffix=col[:3])
  print("One hot vectorizing : ",col,
        "| NaN values : ",categorical_df.isna().apply(lambda x: x*1).sum().sum(),
        "| Number of different categories : ",len(categorical_df.columns),
        "| Shape reduced ? ",categorical_df.shape,oneh_quotebank_brexit.shape)
  # Drop the categorical column
  oneh_quotebank_brexit.drop(columns=col,inplace=True)
  # Check for NaN values
  print("Any NA in the final dataframe: ",oneh_quotebank_brexit.isna().apply(lambda x: x*1).sum().sum())

print("Shape of the final data frame",oneh_quotebank_brexit.shape)
print("Any NA in the final dataframe: ",oneh_quotebank_brexit.isna().apply(lambda x: x*1).sum().sum())

In [ ]:
# this cell is better to remove it at the end of the work

file_onehot = "Brexit_datas/one_hot_quotebank_brexit.json.bz2"

if not exists(file_onehot):
    oneh_quotebank_brexit.to_json(file_onehot)
else:
    oneh_quotebank_brexit = pd.read_json(file_onehot,compression="bz2")
    
oneh_quotebank_brexit.head(5)

<a id='clustering_task'></a>

# Quotations and speakers clustering

The last preprocessing step consists of clustering the quotations as well as the speakers. This clustering will later be used to create a Recommandation Tool in the context of Brexit. Quotations and speakers that carry similar attributes/ideas will belong to the same cluster. Performing such task can be performed using the following pipeline:
1. The first step is to convert sentences into vectors. This task can be achieved using the [SentenceTransformer](https://www.sbert.net/docs/usage/semantic_textual_similarity.html) deep neural network. The vector obtained from this operation cab be then concatenated with the other existing features (that would be converted to one hot vectors if necessary). The specific pre-trained neural network is [multi-qa-mpnet-base-dot-v1](https://huggingface.co/sentence-transformers/multi-qa-mpnet-base-dot-v1), which attended the maximum score for better reliability.
2. The second step consists in reducing the dimension of the data before applying the clustering algorithm. This task can be achieved using the [Locally Linear Embeddings](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html#sklearn.manifold.LocallyLinearEmbedding) algorithm. This algorithm is considered to be an efficient non-linear dimensionality reduction method.
3. The third step is specific to speaker clustering. Indeed, the vectorization of quotes as well as the reduction of dimensionality are only applied to quotes. Thus, we need to perform an **aggregation** to be able to attribute a vector to each speaker. For each speaker, this aggregation can simply be done by taking the mean of the vectors associated with each of their quotations. 
4. The last step consists in performing the clustering operation. This task can be achieved using [Spectral Clustering](#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html#sklearn.cluster.SpectralClustering) method.

### Locally linear embedding

This algorithm aims at preserving the neighbouring points. The process is described as follows: 
- For each point, its nearest neighbors are determined. 
- Then it tries to project the new point in the embedded space such that its neighbors are preserved
This spectral dimensionality reduction technique is non-linear, fast and reliable enough to handle big and complex dataset.

## Encode (or load) the filtered quotations

In [ ]:
# sentence-tranformer improved encoder
sentence_tranformer_improved = 'multi-qa-mpnet-base-dot-v1'

# Encode quotation if necessary
aug_vector_quotes_file = "Brexit_datas/aug_vector_quotes_%s.csv.gz" % sentence_tranformer_improved

if not exists(aug_vector_quotes_file):
    encoder = SentenceTransformer(sentence_tranformer_improved)
    # Encode quotations
    quotes_encoded = encoder.encode(aug_quotebank_brexit['quotation'].values, convert_to_numpy=True, show_progress_bar=True)
    # Convert to df
    quotes_df = pd.DataFrame(quotes_encoded, index = aug_quotebank_brexit.index)
    # Add significant columns
    quotes_df["speaker"] = aug_quotebank_brexit["speaker"].values
    # Export into a compressed format
    quotes_df.to_csv(aug_vector_quotes_file)
    
else:
    # Read the file
    quotes_df = pd.read_csv(aug_vector_quotes_file,index_col=0,compression="gzip")


quotes_df.head()

<a id='One-hot'></a>

## Compose the vectorized dataframe

1. Concatenate the one-hot speaker informations `oneh_cluster_df` with the vectorized quotations `quotes_df`
2. Convert sentiment score into signed integer format: "Positive" = 1, "Negative" = -1, "Neutral" = 0
3. Ensure the values in the data frame are purely numerical
4. Average all rows matching the same speaker 
5. Set `speaker`, `period`, `numOccurrences` and `sentiment_score` as MultiIndex, hence exclude these parameters from the clustering analysis
6. Normalize dataset by row in order to make all variables comparable


In [ ]:
"""
    Compose a vectorized dataframe
"""

# restrict dataframe to the one needed
columns_to_drop = ["Age", "date", "quoteID","qids","phase","probas","urls","date_of_birth","label"]
oneh_cluster_df = oneh_quotebank_brexit.drop(columns=columns_to_drop)

# normalize quotations vector
if 'speaker' in quotes_df.columns:
    quotes_df.drop(columns=['speaker'], inplace=True) # speaker is not needed, already contained in oneh_cluster_df
quotes_df = standardize(quotes_df)

# Merge the two data frames
cluster_full_df = pd.concat([oneh_cluster_df.drop(columns="quotation"), quotes_df.loc[oneh_cluster_df.index]],axis = 1)

# convert sentiment_score to int format
cluster_full_df['sentiment_score'] = cluster_full_df['sentiment_score'].apply(sentiment_to_int)

# average over the same speaker and period
# treat each period differently
cluster_full_df = cluster_full_df.groupby(['speaker', 'period']).agg(np.mean)

# adjust numerical values
cluster_full_df.numOccurrences = cluster_full_df.numOccurrences.astype(int)
cluster_full_df.sentiment_score = cluster_full_df.sentiment_score.round()

# set indexes
cluster_full_df.set_index('numOccurrences', append=True, drop=True, inplace=True)
cluster_full_df.set_index('sentiment_score', append=True, drop=True, inplace=True)

# normalize entire dataset
cluster_full_df = standardize(cluster_full_df)

cluster_full_df.head()

## First analysis aggregation per time period
   - Group by period
   - Apply a 2D Locally Linear Embedding aggregation and visualize
   - Detect outliers with a DBSCAN clustering and filter them

In [ ]:
def embed_dbscan(period_cluster_df):
    
    # embed data
    print("Size of the data which has been embedded: ", period_cluster_df.shape, ", period: ", period_cluster_df.name)
    embedder = LocallyLinearEmbedding(n_components=2, n_neighbors=35, max_iter=500, method='standard', eigen_solver='arpack')
    data_np_emb = embedder.fit_transform(period_cluster_df)
    print("Embedded done, estimated reconstruction error: ", embedder.reconstruction_error_)
    
    # standardize X,Y distribution for embedded data
    std = standardize(data_np_emb)
    
    # apply DBSCAN clustering and detect outliers
    print("\nClustering 2D sample")
    dbscanner = DBSCAN(eps = 1.5, min_samples=10).fit(std)
    print("Outliers found: ", len(std[dbscanner.labels_ == -1]))
    
    std = std.transpose()
    
    period_cluster_df['x'] = std[0]
    period_cluster_df['y'] = std[1]
    period_cluster_df['label'] = dbscanner.labels_
    
    return period_cluster_df.loc[:, ['x', 'y', 'label']]

# embed an label using dbscan    
embedded_df = cluster_full_df.groupby(level=1, sort=False).apply(embed_dbscan)


# plot
fig,axes = plt.subplots(len(period_labels)//2,2, figsize=[15,8])

for ax, period in zip(axes.flatten()[0:len(period_labels)], period_labels):
                      
    period_embedded = embedded_df.xs(period, level=1)
                      
    # plot 
    ax.set_title('Period: %s' % period)
    
    for label in set(period_embedded.label):
        X = period_embedded.loc[period_embedded.label == label, ['x']]
        Y = period_embedded.loc[period_embedded.label == label, ['y']]
        label = "Label %d" % label if label != -1 else "Outliers"
        ax.scatter(X, Y, label=label)
    
    ax.legend()
    

# remove outliers
cluster_filtered_df = cluster_full_df.drop(embedded_df[embedded_df.label == -1].index)

plt.show()

<a id='Clustering'></a>

## Clustering classification (by Brexit period)

   - Embed data into a three dimensional space.
   - Visualize *Silhouette* score and *Calinski Harabasz* score and check for a common optimal number of clusters for each period.
   - Finally, apply clustering with a common reasonable parameter defined by `clustering_number`.
    

### Embed into three dimensions

In [ ]:
global_embed_seed = np.random.seed()

def threedim_embedding(period_df, neighb, iters):
    out_tensor = LocallyLinearEmbedding(n_components=3, n_neighbors=neighb, max_iter=iters, method='standard', eigen_solver='arpack', random_state=global_embed_seed).fit_transform(period_df).transpose()
    
    period_df['x'] = out_tensor[0]
    period_df['y'] = out_tensor[1]
    period_df['z'] = out_tensor[2]
    
    return period_df.loc[:,['x', 'y', 'z']]

cluster_emb_df = cluster_filtered_df.groupby(level=1).apply(threedim_embedding, 25, 1000)
cluster_emb_df.head(2)

### Grid search for optimal number of clusters

In [ ]:
# Number of clusters to be identified 
nb_clusters = range(6,30)


cluster_scores = pd.DataFrame()
clustering_model = lambda n: SpectralClustering(n, n_jobs=4, eigen_solver='arpack', affinity='nearest_neighbors', assign_labels='discretize')

for period in period_labels:
    
    period_df = cluster_emb_df.xs(period, level=1)
    
    print("\nComputing for period: ", period)
    for n_clust in nb_clusters:
        
        # define model
        model = clustering_model(n_clust)
        model = model.fit(period_df)
        
        scores = pd.DataFrame({'cluster_nb' : n_clust, \
                               'silhouette' : metrics.silhouette_score(period_df, model.labels_, metric='euclidean'), \
                               'calinski_harabasz' : metrics.calinski_harabasz_score(period_df, model.labels_), \
                               'period' : period}, index=[0])
                              
        cluster_scores = cluster_scores.append(scores, ignore_index=True)
        
# Plot silhouette score
fig = px.line(cluster_scores, x="cluster_nb", y="silhouette", range_y=[0,0.35], title='Silhouette score for different number of clusters', color="period")
fig["layout"].pop("updatemenus")
fig.show()

# Plot Calinski Harabasz
fig_other = px.line(cluster_scores, x="cluster_nb", y="calinski_harabasz", log_y=True, title='Calinski Harabasz score for different number of clusters', color="period")
fig_other["layout"].pop("updatemenus")
fig_other.show()

### Apply clustering for each period

In [ ]:
clustering_number = 9


def cluster_data(period_df):
    
    model = clustering_model(clustering_number).fit(period_df)
    period_df['Cluster'] = model.labels_
    
    return period_df
    
clustered_df = cluster_emb_df.groupby(level=1, sort=False).apply(cluster_data)

### Visualize embedded optimum clustering results
    - Embed data into two dimensions using Locally Linear Embedding
    - Plot data a first result

In [ ]:
global_embed_seed = np.random.seed()

def final_embed_2d(period_df):
    embedded = LocallyLinearEmbedding(n_components=2, n_neighbors=35, max_iter=1000, method='standard', eigen_solver='arpack', random_state=global_embed_seed).fit_transform(period_df.drop(columns=['Cluster'])).transpose()
    
    period_df['x'] = embedded[0]
    period_df['y'] = embedded[1]
    
    period_df.drop(columns=['z'], inplace=True)
    
    return period_df

    
# embed in 2D
clustered_2d_df = clustered_df.groupby(level=1, sort=False, as_index=False).apply(final_embed_2d)

In [ ]:
# plot
clustered_2d_df_plot = clustered_2d_df.groupby(level=1, sort=False, as_index=False).apply(lambda df: df.sort_values(by=['Cluster']))
clustered_2d_df_plot.Cluster = (clustered_2d_df_plot.Cluster + 1).astype(str)

fig = px.scatter(clustered_2d_df_plot.reset_index(), x="x", y="y", \
                 animation_frame="period", \
                 size = "numOccurrences", \
                 size_max=50, color="Cluster", \
                 hover_name="speaker", \
                 width=800, height=800, opacity=0.9, \
                 title = 'Clustering of speakers vectorized properties shown for each period of Brexit.')

fig["layout"].pop("updatemenus")
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(title = {'text': "Clustering of speakers vectorized properties shown for each period of Brexit.",
                'y':0.1,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'bottom',"font":{"family":"Helvetica","size":18}},
                legend={"font":{"family":"Helvetica","size":16},
                "orientation":"v"},
                sliders = [{'currentvalue.visible' :False, 'font' : {"family":"Helvetica","size":18} }],
                paper_bgcolor="white",plot_bgcolor="white")

fig['layout']['sliders'][0]["y"] = 1.32

fig.show()
if not exists("Plotly_html/clustering_brexit.html"):
    fig.write_html("Plotly_html/clustering_brexit.html",config={"responsive":True})

### Period coherence check

Check that the speakers remain in the same cluster among periods.
Determine which consecutive period clusters are similar in the number of shared speakers and connect them in order to get the correct labelling in the period transition.
The following scheme resumes it for the case of four different clusters and three periods.

![Alt text](Images/reordering_cluster.png "Re-ordering")

This process gives coherence to the animation as the *Locally linear embedding* introduces some randomness and the order information among periods is lost.
The similarity between a cluster an another is computed using the  [Jaccard dissimilarity](https://predictivehacks.com/?all-tips=pairwise-jaccard-similarity) between two clusters containing *speakers* elements.


In [ ]:
# Build a net of next clusters, then re-order labels for animations coherence

import scipy

# distance between two clusters (in speaker similarity terms), defined over [0, 1]
def cluster_distance(cluster_lhs_df, cluster_rhs_df):
    u = set(cluster_lhs_df.speaker.values.tolist())
    v = set(cluster_rhs_df.speaker.values.tolist())
    return 1.0 - len(u.intersection(v)) / len(u.union(v))

# compute mean distance
def determine_next_cluster(cluster_df, period_df):
    
    # identify the minimum distance cluster
    min_result = float('inf')
    next_cluster = None
    
    for cluster in set(period_df.Cluster):
        cluster_j_df = period_df.loc[period_df.Cluster == cluster]
            
        dist = cluster_distance(cluster_df, cluster_j_df)
            
        if dist < min_result:
            min_result = dist
            next_cluster = cluster
                
    return pd.DataFrame({ 'current_cluster' : cluster_df.name,  'min_distance' : min_result, 'next_cluster' : next_cluster}, index=[0])


cluster_label_path = pd.DataFrame()
    
# apply pairwise distance check among consecutive periods
for i in range(len(period_labels) - 1):
    print("Treating period: ", period_labels[i])
    period_df_curr = clustered_2d_df.xs(period_labels[i], level=1).reset_index()
    period_df_next = clustered_2d_df.xs(period_labels[i+1], level=1).reset_index()
    
    update_cluster_table = period_df_curr.groupby('Cluster', sort=False).apply(determine_next_cluster, period_df_next)
    
    if i == 0:
        cluster_label_path[period_labels[i]] = update_cluster_table['current_cluster'].values
        
    cluster_label_path[period_labels[i+1]] = update_cluster_table['next_cluster'].values
    
cluster_label_path = cluster_label_path.astype(int) - 1
cluster_label_path

### Clustering analysis vs. Sentiment score analysis

For each cluster, check whether the speakers share the same sentiment about Brexit.
This analysis will find whether the *properties* of the speakers and their related *quotations* have a strict correlation with the *sentiment score*. 

Let define a score which measures that correlation which is given by:

$$ \rho_{CS} = \frac{\# correctly\_clustered\_speakers}{\# total\_speakers}$$

If $\rho_{CS}$ is close to one, than the properties are fully correlated.
The number of *correctly clustered speakers* is estimated by taking a binary classification on the probability the a speaker $s_i$ belongs to a cluster $C_k$, knowing that the *sentiment score* of $s_i$ is $\alpha \in \{-1, 0, 1\}$.
This decision boundary is given by a threshold on the estimated probability.

In [ ]:
# cluster sentiment comparison score

def cluster_score_sentiments(single_cluster_df):
    # get sentiment score normalized occurrences (= probability) 
    unique_prob = single_cluster_df.sentiment_score.value_counts(normalize = True)
    
    # replace sentiment score by it's related probability
    single_cluster_df['cluster_sentiment_probability'] = single_cluster_df.sentiment_score.apply(lambda v: unique_prob[v])
    return single_cluster_df
    

# define cluster sentiment threshold score
cluster_sentiment_threshold = 0.45 # between 0 and 1

cluster_sentiment_similarity_score = 0
clust_sent_similarity_df = pd.DataFrame()


clustered_final_df = clustered_2d_df
    
for period in period_labels:
    period_df = clustered_final_df.xs(period, level=1).reset_index()
    period_df = period_df.groupby('Cluster').apply(cluster_score_sentiments)

    # apply threshold binary classification
    period_df.cluster_sentiment_probability = period_df.cluster_sentiment_probability > cluster_sentiment_threshold
    
    score_by_cluster = pd.DataFrame()
    
    score_by_cluster['score'] = period_df.groupby('Cluster', sort=False).cluster_sentiment_probability.agg(np.mean)
    
    score_by_cluster['Cluster'] = period_df.groupby('Cluster', sort=False, as_index=False).agg(lambda x: x.name).Cluster
    score_by_cluster['period'] = period
    
    cluster_sentiment_similarity_score += score_by_cluster.score.mean()
    clust_sent_similarity_df = clust_sent_similarity_df.append(score_by_cluster)

cluster_sentiment_similarity_score /= len(period_labels)
    
print("Total score: ", cluster_sentiment_similarity_score)

# Plot  score for each cluster
fig = px.line(clust_sent_similarity_df.reset_index(drop=True).groupby('period').apply(lambda x: x.sort_values(by=['Cluster'])), x="Cluster", y="score", range_y=[0,1], title='Score of comparison between clustering assignment and speaker sentiment score', color="period")
fig["layout"].pop("updatemenus")
fig.show()

<a id='Results'></a>

# Generate the results for the final story

<a id='Statistics'></a>

## General Statistics

Now that we had preprocessed the datas let's have a look at different basic statistics to explore deeply the dataset. Let's first look at the distribution of the quotations accross time.

Some basic statistics: 

In [ ]:
# Some numbers 
nb_quotations = len(quotebank_brexit)
nb_speakers = len(pd.unique(oneh_quotebank_brexit.speaker))
nb_countries = len(unique_values["nationality"])
print("Number of quotations :",nb_quotations)
print("Number of speakers :",nb_speakers)
print("Number of countries :",nb_countries)

Evolution of the number of quotations about Brexit accross time

In [ ]:
# windows size to compute the mooving average 
windows_avg_quotations = 9

# generate a data range to complete missing values
date_range = pd.date_range(start=quotebank_brexit.date.dt.date.min(),end=quotebank_brexit.date.dt.date.max())
quotes_count = pd.DataFrame(date_range,columns=["date"])
# Count the number of quotes per date on the original dataset
count_quotebank = quotebank_brexit.date.dt.date.to_frame().groupby("date").size().reset_index(name="Count")
count_quotebank["date"] = pd.to_datetime(count_quotebank["date"])
# Merge the data range with the original data set
quotes_count = pd.merge(quotes_count,count_quotebank,on="date",how="left")
# date that do not have count then set it to 0 (no quotation recorded)
quotes_count.loc[quotes_count["Count"].isna(),"Count"] = 0
# Compute the mooving average 
quotes_count["moo_avg"] = quotes_count["Count"].rolling(windows_avg_quotations,center=True).mean()
# Only keep quotations after 2016, almost no quotations speaking about brexit before 2016
quotes_count = quotes_count[quotes_count.date.dt.year >= 2016]

# Initialize the figure
fig = go.Figure()

# Plot the histogram of number of quotations
fig.add_trace(go.Histogram(x=quotebank_brexit[quotebank_brexit.date.dt.year >= 2016].date.dt.date.values,xbins=dict(
                      start=str(quotes_count.date.dt.date.min()),
                      end=str(quotes_count.date.dt.date.max()),
                      size='D'),autobinx=False, opacity=0.3, marker=dict(color="blue"),name="Number of quotations"))

# Plot the moving average 
fig.add_trace(go.Scatter(x=quotes_count.date,y=quotes_count["moo_avg"],
              mode="lines",marker=dict(color="blue"),name=str(windows_avg_quotations) +"-days averaged"))

# Set the font and other styles 
fig.update_layout(paper_bgcolor="white",plot_bgcolor="white",
                    legend=dict(
                    font={"family":"Helvetica"},
                    orientation="h",
                    yanchor="bottom",
                    y=1.1,
                    xanchor="left",
                    x=0),
                   xaxis=dict(rangeslider=dict(
                    visible=True
                    )))

fig.update_layout(title = {'text': "Number of quotations about Brexit",
                 'y':0.95,
                 'x':0.5,
                 'xanchor': 'center',
                 'yanchor': 'top',"font":{"family":"Helvetica","size":18}})

# Label axes 
fig.update_yaxes(title=dict(text="Number of quotations",font={"family":"Helvetica"}))
fig.update_xaxes(title=dict(text="Date",font={"family":"Helvetica"}))

# Show that 
fig.show()

if not exists("Plotly_html/quotations_brexit.html"):
    fig.write_html("Plotly_html/quotations_brexit.html",config={"responsive":True})

quotes_count.head()

Let's see the top 20 countries that are providing the most quotations about Brexit

In [ ]:
country_df = oneh_quotebank_brexit.loc[:,unique_values["nationality"]].sum(axis=0).T.to_frame().reset_index()
country_df = country_df.sort_values(by=0,ascending=False)
fig = px.bar(country_df,y=0,x="index",log_y=True)
fig.update_layout(title="Number of quotations about Brexit accross time",
                  xaxis_title="Country",yaxis_title="Count")
fig.show()

Let's see the top sectors that are providing the most quotations on Brexit

In [ ]:
job_df = oneh_quotebank_brexit.loc[:,unique_values["occupation"]].sum(axis=0).T.to_frame().reset_index()
job_df = job_df.sort_values(by=0,ascending=False)
fig = px.bar(job_df,y=0,x="index",log_y=True)
fig.update_layout(title="Number of quotations about Brexit accross time",
                  xaxis_title="Sector",yaxis_title="Count")
fig.show()

Let's see the top 10 speakers that are providing the most quotations on Brexit

In [ ]:
aug_quotebank_brexit.groupby("speaker").size().reset_index(name="count").loc[:,["speaker","count"]].sort_values(by="count",ascending=False).head(10)

In [ ]:
# Define a function that will be used in further studies 
# This function returns a subset of features about quotations
# And this function filters also the dataset according to a year range

def select_by_year(low_year,up_year, col=None):
    # Convert date into year
    year_col = pd.DatetimeIndex(oneh_quotebank_brexit.date).year
    if col is None:
        cols = ["sentiment_score"]
    else:
        cols = list(unique_values[col]) + ["sentiment_score"] # Features selected
    # Filter according to the date and the features
    filter_df = oneh_quotebank_brexit.loc[(year_col >= low_year) & (year_col <= up_year),cols]
    # Group by sentiment score and compute the sum
    filter_df = filter_df.groupby("sentiment_score").sum()
    # count the number of quotation per feature
    count = filter_df.sum(axis=0)
    # Normalize between [0 100] and transpose 
    filter_df = (filter_df * 100/ filter_df.sum(axis=0)).T.reset_index()
    # Add a count column
    filter_df["count"] = count.values
    return filter_df

### Feeling about brexit in the united kingdom

As an introduction, we will first have a look on the evolution of the feeling about Brexit in the more involved country: the United Kingdom. 

In [ ]:
# Get the quotations from United kingdom
UK_df = oneh_quotebank_brexit[oneh_quotebank_brexit["united kingdom"] == 1]
# Years to plot (removed 2015 because not enough data)
years = list(range(2016,2021))

# Negative - Neutral - Positive colors 
color_discrete_map = ['#FF5B5B',"#FFF992",'#7BD787']

# Initialize the figure 
fig = make_subplots(rows=1, cols=len(years), specs=[[{'type':'domain'}]*len(years)])

# Loop over the years 
for i,pie_year in enumerate(years):
    # Get a year index
    year_col = pd.DatetimeIndex(UK_df.date).year
    # Get the right year 
    filter_df = UK_df.loc[year_col == pie_year,["sentiment_score"]]
    # Group by sentiment and count for each 
    filter_df = filter_df.groupby("sentiment_score").size().reset_index(name="Count")
    # Generate the pie_chart
    pie_trace = go.Pie(labels=filter_df.sentiment_score,
                       values=filter_df.Count,name=str(pie_year),
                       title=str(pie_year))
    # Add a trace to the figure 
    fig.add_trace(pie_trace,1,i+1)

# Make a hole, set colors 
fig.update_traces(hole=.4, hoverinfo="label+percent+name",marker = dict(colors=color_discrete_map),textinfo='none')

# Tune the figure 
fig.update_layout(title = {'text': "Semtiment towards Brexit grouped by year [United kingdom]",
                'y':0.1,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'bottom',"font":{"family":"Helvetica","size":18}},
                legend={"font":{"family":"Helvetica","size":16},
                "orientation":"h",
                "yanchor":"top",
                "y":1.2,
                "xanchor":"center",
                "x":0.5})

fig.update_layout(height=400,width=1200)

# fig.show(config={"responsive":True})

# Export the figure 
if not exists("Plotly_html/pie_chart.html"):
    fig.write_html("Plotly_html/pie_chart.html",config={"responsive":True})

UK_df.head()

#### Is there a real difference between 2016 and 2020 ?

We would like to figure out if the feeling about Brexit in the United Kingdom was really different between 2016 and 2020. To achieve this, we will compare the sentiment distributions of 2016 and 2020 using a Pearson's chi-squared test. The null hypothesis of the Chi-Square test is that no relationship exists on the categorical variables in the population; they are independent.

In [ ]:
# Specify which year you want to test 
year_pair = [(2016,2020),(2016,2018)]

for pair in year_pair:
    # Extract the information
    UK_low_df = UK_df.loc[UK_df.date.dt.year == pair[0],["sentiment_score"]].groupby("sentiment_score").size().reset_index(name="Count")
    UK_up_df = UK_df.loc[UK_df.date.dt.year == pair[1],["sentiment_score"]].groupby("sentiment_score").size().reset_index(name="Count")
    observed = np.concatenate([UK_low_df["Count"].values.reshape(1,-1),UK_up_df["Count"].values.reshape(1,-1)],axis=0)
    # Perform kolmogorov smirnov test
    stat_val, p_value, _, _ = stats.chi2_contingency(observed)
    print("Difference between ",pair[1],"and",pair[0])
    print("Student t test : stastics:",stat_val," p-value: ",p_value)


<a id='Country'></a>

## Analyze the way Brexit is perceived in European countries

Recall that the goal is to analyze the way Brexit is perceived by each Europe country based on the sentiment carried by the quotation. Besides, we would like to add the time dimension to this analysis, meaning that we would like to follow the evolution of the overall feelings towards Brexit. To achieve this we will compute for each country the overall feeling about Brexit up to a year (2016 to 2020), we will then merge this into a single dataframe. This final dataframe will then be used to create an animation to follow the brexit sentiment.

#### Get all the information needed for a map plot

In [ ]:
# Load the geojson file that will be required to define the map 
with open("Brexit_datas\country_class\countries.geojson") as file:
    country_gj = geojson.load(file)

# Get the countries and their ISO2 id
current_countries = pd.read_excel("Brexit_datas\country_class\countries.xlsx")

# Remove capital letters and special characters
current_countries.Country = current_countries.Country.str.lower()
current_countries.Country = current_countries.Country.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Map ISO2 id to ISO3 id (we need ISO3 instead of ISO2 because ISO3 is specified in the geojson dataset)
ISO_mapping = pd.read_csv("Brexit_datas\country_class\ISO2_ISO3.csv")
current_countries = pd.merge(ISO_mapping,current_countries,left_on="ISO2",right_on="ISO",how="right").drop(columns=["ISO","ISO2"])

# Remove countries for which ISO3 was not found 
current_countries = current_countries[~current_countries.ISO3.isna()]

# Export the first dataset needed 
if not exists("Brexit_datas/country_class/countries_to_ISO3.csv"):
    current_countries.to_csv("Brexit_datas/country_class/countries_to_ISO3.csv",index=False)

# Extract only relevant features 
country_analysis_df = oneh_quotebank_brexit[list(unique_values["nationality"]) + ["sentiment_score","date"]]
country_analysis_df["date"] = country_analysis_df.date.dt.year

# Define a function that will be used in further studies 
# This function returns a subset of features about quotations
# And this function filters also the dataset according to a year range
def select_by_year_country(low, high, dash_df):
    filter_df = dash_df[(dash_df.date >= low) & (dash_df.date <= high)]
    filter_df.drop(columns="date", inplace=True)
    filter_df = filter_df.groupby("sentiment_score").sum()
    count = filter_df.sum(axis=0)
    filter_df = (filter_df * 100 / filter_df.sum(axis=0)).T.reset_index().rename(columns={"index":"country"})
    filter_df["count"] = count.values
    filter_df = filter_df[filter_df["count"] > 40]
    filter_df["Sentiment"] = -1*filter_df.Negative + filter_df.Positive
    filter_df = pd.merge(filter_df,current_countries,left_on="country",right_on="Country",how="left").drop(columns=["country"])
    return filter_df

keep_only = current_countries[current_countries.Country.isin(country_analysis_df.columns)]
keep_only = list(keep_only["ISO3"])

features = []

for country_geo in country_gj["features"]:
    if country_geo["properties"]["ISO_A3"] in keep_only:
        features.append(geojson.Feature(geometry=country_geo["geometry"], properties=country_geo["properties"]))

feature_collection = geojson.FeatureCollection(features)

with open('myfile.geojson', 'w') as f:
   geojson.dump(feature_collection, f)

# Load the geojson file that will be required to define the map 
with open("myfile.geojson") as file:
    country_gj = geojson.load(file)

os.remove("myfile.geojson")

country_analysis_df.head()

In [ ]:
# Compile the evolution into a single dataframe
full_country_df = select_by_year_country(2016,2016,country_analysis_df)
full_country_df["year"] = 2016
for year in range(2017,2021):
    new_year = select_by_year_country(2016,year,country_analysis_df)
    new_year["year"] = year
    full_country_df = pd.concat([full_country_df,new_year],axis=0)

# Plot the figure with the analysis 
fig = px.choropleth_mapbox(full_country_df, geojson=country_gj, locations='ISO3', color='Sentiment',
                            animation_frame="year",
                            featureidkey="properties.ISO_A3",
                            range_color=[-10,35],
                            color_continuous_scale="RdYlGn",
                            mapbox_style="carto-positron",
                            zoom=2.5, center = {"lat": 48.856613, "lon": 2.352222},
                            opacity=0.6,hover_data=["Country","count"])

# Tune the map 
fig.update_layout(title={'text': "Evolution of the feeling about Brexit per country [2016-2020]",
                             'y':0.02,
                             'x':0.5,
                             'xanchor': 'center',
                             'yanchor': 'bottom'},legend_font_family="Helvetica",legend_xanchor="center",
                             legend_yanchor="bottom",
                             title_font_family="Helvetica",title_font_size=20,height=950)

fig['layout']['updatemenus'][0]["y"] = 1.32
fig['layout']['sliders'][0]["y"] = 1.32
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000

if not exists("Plotly_html/map_anim.html"):
    fig.write_html("Plotly_html/map_anim.html")

full_country_df.head()

<a id='Sector'></a>

## Analyze the way Brexit is perceived in different sectors

Then, we would like to perform a similar analysis, but based on the different sectors (economy, health, art ...). Let see if there are different trends between the different sectors. Again as there is more than 20 sectors we cannot afford to plot all the distributions in a single graph and we would need some filters to compare some given sector. Besides we would to add the time dimension again. 

To do so we need to use dynamic graphs with filters that allows the user to modify the year. This can be achieved using dash plotly application. This application, as well as the associated stuff needed, will be embeded into a **Heroku** server that will actually host the application. We will need to create two things:
- The dataset that will be embedded in the heroku app. This dataset will be called each time the user wants to modify something.
- The script that will drive the application (callbacks, layout and so on ...)

We will then finally send these two files onto a heroku repository and then the application will run and be available online.

#### Generate the dataset that will be embeded into the heroku application

In [ ]:
# Select the right subset of features 
sector_df = oneh_quotebank_brexit[list(unique_values["occupation"]) + ["sentiment_score","date"]]
sector_df["date"] = sector_df.date.dt.year

if not exists("Dash_datas/dash_data_occupation.csv.gz"):
    sector_df.to_csv("Dash_datas/dash_data_occupation.csv.gz",index=False)

sector_df.head()

#### Python script embedded into the heroku app

In [ ]:
import dash
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd

# Initialize the app
app = dash.Dash(__name__)
server = app.server

# Colors that will be displayed on the pie charts 
color_discrete_map = ['#FF5B5B',"#FFF992",'#7BD787',"#000000"]

# Define a function that will be used in further studies 
# This function returns a subset of features about quotations
# And this function filters also the dataset according to a year range
def select_by_year_occupation(low,high,dash_df):
    filter_df = dash_df[(dash_df.date >= low) & (dash_df.date <= high)]
    filter_df.drop(columns="date",inplace=True)
    filter_df = filter_df.groupby("sentiment_score").sum()
    count = filter_df.sum(axis=0)
    filter_df = filter_df.T.reset_index().rename(columns={"index":"occupation"})
    filter_df["count"] = count.values
    return filter_df

# Get the datas 
dash_data = pd.read_csv("Dash_datas/dash_data_occupation.csv.gz",compression="infer")
P_val_df = pd.read_csv("Dash_datas/dash_datas_sector_p_values.csv",index_col=0)

# Get the list of the occupations
occupations = list(dash_data.columns[:-2])

# Define the marks for the slider 
marks_slider = {year:{"label" : str(year),
               "style" : {"color":"black","font-family":"Helvetica","font-size": 18}} 
               for year in range(dash_data.date.min()+1,dash_data.date.max()+1)}

# Define the options for the dropdown
options = [{"label": occup, "value": occup,
            "style" : {"color":"black","font-family":"Helvetica","font-size": 18}} 
            for occup in list(occupations)]

# Define the layout of the application
app.layout = html.Div([
        html.Div([html.P("Year",style={"display": "flex",
                                "font-family":"Helvetica",
                                "font-size": 18,
                                "align-items": "center",
                                "justify-content": "center"}),
                            dcc.RangeSlider(
                                id='range-slider',
                                min=dash_data.date.min()+1, max=dash_data.date.max(), step=1,
                                marks=marks_slider,
                                value=[dash_data.date.min()+1,dash_data.date.max()])],
                            style = {"width":"60%","margin-bottom":25,"margin-left":"20%","width":"60%","margin-right":"20%"}),
        html.Div([dcc.Dropdown(id='drop-down',multi=True,
                               style={"color":"black","font-family":"Helvetica","font-size": 18},
                               options=options,value=["Art","Health","Economy"])],
                 style = {"width":"60%","margin-left":"20%","margin-right":"20%"}),
        html.Div([dcc.Graph(id="pie-chart",responsive=True)])
    ])


# function that will update pies and drop down values 
@app.callback(Output("pie-chart", "figure"),
              [Input("range-slider","value"),Input("drop-down","value")])
def update_pies(slider_range,values_selected):
    # get the year range wanted 
    low, high = slider_range
    # Filter according to the year 
    sector_analysis = select_by_year_occupation(low,high,dash_data)
    # Filter by the sectors that are specified in the dropdown list
    sector_analysis = sector_analysis[sector_analysis["occupation"].isin(values_selected)]
    # labels for graph
    labels = list(sector_analysis.columns[-4:-1])
    # Initialize the figure 
    fig = make_subplots(rows=1,cols=len(sector_analysis),specs=[[{"type":"domain"}]*len(sector_analysis)])
    start = 1
    # Sort by count 
    sector_analysis.sort_values(by="count",ascending=False,inplace=True)
    # Loop over the sector analysis rows 
    for index, row in sector_analysis.iterrows():
        # If count lower than 40 then not enough data to well represent the sector
        if row["count"] < 40:
            trace = go.Pie(labels=labels + ["Not enough data"],
                           values=[0,0,0,1],
                           name=row["occupation"],
                           title=row["occupation"])
        else:
            trace = go.Pie(labels=labels,
                            values=row[labels].values,
                            name=row["occupation"],
                            title=row["occupation"])
        # Add the trace 
        fig.add_trace(trace,row =1,col=start)
        start += 1
    # Tune the figure 
    fig.update_traces(hole=.4, hoverinfo="label+percent+name",marker = dict(colors=color_discrete_map),textinfo='none')
    fig.update_layout(title = {'text': "Semtiment towards Brexit by sector",
                        'y':0.05,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'bottom',"font":{"family":"Helvetica","size":20}},
                        legend={"font":{"family":"Helvetica","size":16},
                        "orientation":"h",
                        "yanchor":"top",
                        "y":1.2,
                        "xanchor":"center",
                        "x":0.5})
    return fig

if __name__ == '__main__':
    app.run_server()

#### Is there a real difference between sectors ?

We would like to figure out if the feeling about Brexit is really different from one sector to another, idest is there a statistically significant difference between two sectors. We selected a subset of pairs of sectors for which we would like to test that. To achieve this, we will compare the sentiment distributions between sectors using a Pearson's chi-squared test. The null hypothesis of the Chi-Square test is that no relationship exists on the categorical variables in the population; they are independent.

In [ ]:
# Specify which year you want to test 
age_categories = list(sector_df.columns[:-2])
p_value_matrix = []

for age1 in age_categories:
    p_value_line = []
    for age2 in age_categories:
        # Extract the information
        first_category = sector_df.loc[sector_df[age1] == 1,["sentiment_score"]]
        second_category = sector_df.loc[sector_df[age2] == 1,["sentiment_score"]]
        # Convert to numerical values (discrete ones)
        first_category.loc[:,"sentiment_score"] = first_category.sentiment_score.apply(lambda x : 1*(x == "Positive") - 1*(x == "Negative"))
        second_category.loc[:,"sentiment_score"] = second_category.sentiment_score.apply(lambda x : 1*(x == "Positive") - 1*(x == "Negative"))
        # Perform kolmogorov smirnov test
        stat_val, p_value = stats.ttest_ind(first_category.sentiment_score.values, second_category.sentiment_score.values,equal_var=False)
        p_value_line.append(p_value)
        # print("Difference between ",pair[1],"and",pair[0])
        # print("Kolmogorov smirnov test : stastics:",stat_val," p-value: ",p_value)
    p_value_matrix.append(p_value_line)

P_val_df = pd.DataFrame(p_value_matrix,columns=age_categories,index=age_categories)

if not exists("Dash_datas/dash_datas_sector_p_values.csv"):
    P_val_df.to_csv("Dash_datas/dash_datas_sector_p_values.csv")

sector_df.head()

# Heat map figure now for p-values 
fig_heat_sheet = px.imshow(P_val_df,range_color=[0,1],labels=dict(color="p-value"))
fig_heat = go.Figure()
for trace in fig_heat_sheet["data"]:
    fig_heat.add_trace(trace)
fig_heat.update_layout(title = {'text': "Result of the Welch's t-test (p-values)",
                        'y':0.95,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',"font":{"family":"Helvetica","size":20}},
                        legend={"font":{"family":"Helvetica","size":10},
                        "orientation":"h",
                        "yanchor":"top",
                        "y":1.2,
                        "xanchor":"center",
                        "x":0.5},height=600)

fig_heat.write_image("Plotly_html/heat_map_sector.svg")



<a id='Age'></a>

# Age analysis

It is common to hear that Brexit is mostly due to elderly people and that young people are mostly against Brexit. Let try to see if such a feeling is confirmed by our datas. We will study the feeling about Brexit for each age category. We will here again use a dynamic graph, hence we will again follow the same procedure as decribed below.

To do so we need to use dynamic graphs, this can be achieved using dash plotly application. This application, as well as the associated stuff needed, will be embeded into a **Heroku** server that will actually host the application. We will need to create two things:
- The dataset that will be embedded in the heroku app. This dataset will be called each time the user wants to modify something.
- The script that will drive the application (callbacks, layout and so on ...)

We will then finally send these two files onto a heroku repository and then the application will run and be available online.

#### Generate the dataset that will be embeded into the heroku application

In [ ]:
# Get relevant features 
age_df = oneh_quotebank_brexit.loc[:,["speaker","Age","sentiment_score","united kingdom"]]
# Compute an overall sentiment score 
age_df["score"] = age_df["sentiment_score"].apply(lambda sent: 1*(sent == "Positive") - 1*(sent == "Negative"))
# Drop useless columns 
age_df.drop(columns=["sentiment_score"],inplace=True)
# Take the mean for each speaker
age_df = age_df.groupby("speaker").mean()
# Remove any speakers for which we do not have age information
age_df = age_df[~age_df["Age"].isna()]

# Convert continuous sentiment score into categorical values 
def reverse_sentiment(score, threshold = 0.05):
    if score > threshold:
        return "Positive"
    elif score < -threshold:
        return "Negative"
    else:
        return "Neutral"

# convert to categorical
age_df["sentiment"] = age_df["score"].apply(reverse_sentiment)
# Categorize by ages - size bin = 10
age_df["Age"] = pd.cut(age_df.Age,bins=list(range(20,101,10)))
raw_age_df = age_df.reset_index().drop(columns=["speaker","score"])
UK_age_df = age_df[age_df["united kingdom"] == 1]
# One hot vectorize that 
age_df = pd.get_dummies(age_df.reset_index().drop(columns=["speaker","score","united kingdom"]).set_index("Age")).reset_index()
UK_age_df = pd.get_dummies(UK_age_df.reset_index().drop(columns=["speaker","score","united kingdom"]).set_index("Age")).reset_index()
# Group by age category and sum 
age_df = age_df.groupby("Age").sum()
age_df.columns = ["Negative","Neutral","Positive"]
UK_age_df = UK_age_df.groupby("Age").sum()
UK_age_df.columns = ["Negative","Neutral","Positive"]

# Export the datas 
if not exists("Dash_datas/dash_data_age.csv"):
    age_df.to_csv("Dash_datas/dash_data_age.csv")
if not exists("Dash_datas/dash_data_UK_age.csv"):
    UK_age_df.to_csv("Dash_datas/dash_data_UK_age.csv")

UK_age_df.head()

#### Python script embedded into the heroku app

In [ ]:
import dash
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd

# Initialize the app
app = dash.Dash(__name__)
server = app.server

# Colors to be used in the bar chart
color_discrete_map = ['#FF5B5B',"#FFF992",'#7BD787']

# Import the datas
age_df = pd.read_csv("Dash_datas/dash_data_age.csv",index_col="Age")
age_df_percentage = (age_df*100) / age_df.sum(axis=1).values.reshape(-1,1)

# Import the datas
UK_age_df = pd.read_csv("Dash_datas/dash_data_UK_age.csv",index_col="Age")
UK_age_df_percentage = (UK_age_df*100) / UK_age_df.sum(axis=1).values.reshape(-1,1)

# X values for the bar chart
x_values = list(age_df.index.astype(str))
x_values = ["Age " + age for age in x_values]

# Options available 
graph_type = ["Percentage","Absolute"]

options = [{"label": gtype, "value": gtype,
            "style" : {"color":"black","font-family":"Helvetica","font-size": 18}} 
            for gtype in list(graph_type)]

UK_opt = ["World","United kingdom"]

options_UK = [{"label": gtype, "value": gtype,
                "style" : {"color":"black","font-family":"Helvetica","font-size": 18}} 
                for gtype in list(UK_opt)]

# Define the layout of the application
app.layout = html.Div([html.Div([
                        html.Div([dcc.RadioItems(id='drop-down',
                                       options=options,
                                       labelStyle={'display': 'inline-block',
                                                "color":"black",
                                                "font-family":"Helvetica",
                                                "font-size": 18},
                                       value="Percentage")],
                                    style={"display":"inline-block","margin-left":"5%"}),
                        html.Div([dcc.RadioItems(id='drop-down-UK',
                                       options=options_UK,
                                       labelStyle={'display': 'inline-block',
                                                "color":"black",
                                                "font-family":"Helvetica",
                                                "font-size": 18},
                                       value="World")],
                                    style={"display":"inline-block","margin-left":"5%"}),]),
                        dcc.Graph(id="bar-chart",responsive=True)
    ]
)


# Update the bar chart
@app.callback(Output("bar-chart", "figure"),
              [Input("drop-down","value"),Input("drop-down-UK","value")])
def update_bars(value_selected,value_UK):
    # Percentage or absolute 
    if value_UK == "United kingdom":
        df = UK_age_df
    else:
        df = age_df
    if value_selected == "Percentage":
        df = (df*100) / df.sum(axis=1).values.reshape(-1,1)

    fig = go.Figure()

    # for each type of sentiment 
    for i,col in enumerate(df.columns):
        trace = go.Bar(name = col, x=x_values, y = df[col].values, 
                       marker=dict(color=color_discrete_map[i]))
        fig.add_trace(trace)
    
    # Tune the figure
    fig.update_layout(paper_bgcolor="white",plot_bgcolor="white",
                      title = {'text': "Perception of Brexit by the different age groups",
                      'y':0.03,
                      'x':0.5,
                      'xanchor': 'center',
                      'yanchor': 'bottom',"font":{"family":"Helvetica","size":22}},
                      legend={"font":{"family":"Helvetica","size":16},
                      "orientation":"h",
                      "yanchor":"top",
                      "y":1.2,
                      "xanchor":"center",
                      "x":0.5})
    
    # Do not show y axis if it is percentage 
    if value_selected == "Percentage":
        fig.update_yaxes(title=dict(text="Percentage",font={"family":"Helvetica"}))
    else:
        fig.update_yaxes(title=dict(text="Number of speakers",font={"family":"Helvetica"}))
    return fig

if __name__ == '__main__':
    app.run_server()

#### Is there a real difference between age categories ?

Let see if the rumor previously mentionned is true or not, idest let check if there exists a statistically significant difference between two age category. We selected a subset of pairs of age categories for which we would like to test that. To achieve this, we will compare the sentiment distributions between two age categories using a Pearson's chi-squared test. The null hypothesis of the Chi-Square test is that no relationship exists on the categorical variables in the population; they are independent.

In [ ]:
# Specify which year you want to test 
raw_age_df = raw_age_df[~raw_age_df.Age.isna()]
raw_age_df.sort_values(by="Age",inplace=True)
raw_age_df.loc[:,"Age"] = raw_age_df.loc[:,"Age"].astype(str)
age_categories = list(pd.unique(raw_age_df["Age"]))
p_value_matrix = []
p_value_matrix_UK = []

for age1 in age_categories:
    p_value_line = []
    p_value_line_UK = []
    for age2 in age_categories:
        # Extract the information
        first_category = raw_age_df.loc[raw_age_df["Age"] == age1,["sentiment","united kingdom"]]
        second_category = raw_age_df.loc[raw_age_df["Age"] == age2,["sentiment","united kingdom"]]
        # Convert to numerical values (discrete ones)
        first_category.loc[:,"sentiment"] = first_category.sentiment.apply(lambda x : 1*(x == "Positive") - 1*(x == "Negative"))
        second_category.loc[:,"sentiment"] = second_category.sentiment.apply(lambda x : 1*(x == "Positive") - 1*(x == "Negative"))
        # Perform kolmogorov smirnov test
        stat_val, p_value = stats.ttest_ind(first_category.sentiment.values, second_category.sentiment.values,equal_var=False)
        stat_val, p_value_UK = stats.ttest_ind(first_category[first_category["united kingdom"] == 1].sentiment.values, 
                                              second_category[second_category["united kingdom"] == 1].sentiment.values,equal_var=False)
        p_value_line.append(p_value)
        p_value_line_UK.append(p_value_UK)
        # print("Difference between ",pair[1],"and",pair[0])
        # print("Kolmogorov smirnov test : stastics:",stat_val," p-value: ",p_value)
    p_value_matrix.append(p_value_line)
    p_value_matrix_UK.append(p_value_line_UK)

P_val_df = pd.DataFrame(p_value_matrix,columns=age_categories,index=age_categories)
P_val_df_UK = pd.DataFrame(p_value_matrix_UK,columns=age_categories,index=age_categories)

if not exists("Dash_datas/dash_datas_age_p_value.csv"):
    P_val_df.to_csv("Dash_datas/dash_datas_age_p_value.csv")
if not exists("Dash_datas/dash_datas_age_UK_p_value.csv"):
    P_val_df_UK.to_csv("Dash_datas/dash_datas_age_UK_p_value.csv")

fig_heat = make_subplots(rows=1,cols=2,subplot_titles=("Result of the Welch's t-test (p-values) <br>[United Kingdom]",
                                                           "Result of the Welch's t-test (p-values)"))

# Heat maps 
fig_world = px.imshow(P_val_df,range_color=[0,1],labels=dict(color="p-value"))
fig_UK = px.imshow(P_val_df_UK,range_color=[0,1],labels=dict(color="p-value"))
# Add the trace 
for trace in fig_UK["data"]:
    fig_heat.add_trace(trace,1,1)
for trace in fig_world["data"]:
    fig_heat.add_trace(trace,1,2)

fig_heat['layout']['xaxis']['title']['font']['family'] = 'Helvetica'
fig_heat['layout']['xaxis2']['title']['font']['family'] = 'Helvetica'
fig_heat['layout']['yaxis']['title']['font']['family'] = 'Helvetica'
fig_heat['layout']['yaxis2']['title']['font']['family'] = 'Helvetica'
fig_heat['layout']['titlefont']['family'] = 'Helvetica'

fig_heat.update_layout(width=1200,height=600)

fig_heat.write_image("Plotly_html/heat_map_age.svg")

<a id='Gender'></a>

## Gender Analysis

In the same spirit it was done before, let see if the feeling about Brexit is a function of the gender. 

In [ ]:
# Select gender columns and compute for each of them some values 
gender_df = select_by_year(2016,2020,"gender").rename(columns={"index":"gender"})
# List of distinct genders
x_values = list(gender_df.gender)

# Colors for the bar chart 
color_discrete_map = ['#FF5B5B',"#FFF992",'#7BD787']
cols = ["Negative","Neutral","Positive"]

# Initialize the figure 
fig = go.Figure()

# Add the histograms for each gender 
for i,col in enumerate(cols):
    trace = go.Bar(name = col, x=x_values, y = gender_df[col].values, 
                    marker=dict(color=color_discrete_map[i]))
    fig.add_trace(trace)

# Tune the figure layout 
fig.update_layout(paper_bgcolor="white",plot_bgcolor="white",
                  title = {'text': "Difference in perception of Brexit by people of different gender",
                  'y':0.03,
                  'x':0.5,
                  'xanchor': 'center',
                  'yanchor': 'bottom',"font":{"family":"Helvetica","size":20}},
                  legend={"font":{"family":"Helvetica","size":16},
                  "orientation":"h",
                  "yanchor":"top",
                  "y":1.2,
                  "xanchor":"center",
                  "x":0.5})

# fig.show()

if not exists("Plotly_html/gender_bar.html"):
    fig.write_html("Plotly_html/gender_bar.html")

gender_df.head()

<a id='Stocks'></a>

## Correlation with stocks

We would like to have more insights on how the Brexit influenced the Economy. To do so we will look at the top 100 british stock auctions also known at FTSE100 and try to see if they are correlated to the Brexit. To assess the correlation between Brexit and stock auction we followed the procedure described below: 
- We could actually measure the turmoil due to Brexit by looking at the evolution of the number of quotation accross time. A huge number of new quotations in a short time means that there is a a high turmoil due to Brexit. Thus it seems natural to estimate the Brexit turmoil through the derivative of the number quotations with respect to the time. Instead of taking the derivative of the original curve, we would rather take the derivative of the cumulative sum of the number of quotations with respect to time, as a decrease in the number of quotations would not have a real sense.
- In the same spirit, regarding stock auctions, we are not really interested in the absolute value of them. We would rather prefer taking the derivative that tells you if the stock auction rises or falls. We then want to see if these rises or falls are correlated with Brexit turmoil. Besides, to simplify the things, we will take the absolute value of the derivative as Brexit may cause negative as well as positive effects.
- Finally we will them compute the pearson coefficient correlation coefficient that will give us an insights on how Brexit is correlated with stock auction.

Before computing any derivative, it may be important to recall that we are performing some signal processing on the original time series (in a nutshell mooving average) to get ride of an eventual noise. At the end we will provide an interactive dashboard that will allow the user to see the correlation between Brexit and some stock auction. We will only keep the ones with a sufficient pearson correlation coefficient and sufficient low p-value. We will here again use a dynamic graph to generate such a dashboard, hence we will again follow the same procedure as decribed below:

To do so we need to use dynamic graphs, This can be achieved using dash plotly application. This application, as well as the associated stuff needed, will be embeded into a **Heroku** server that will actually host the application. We will need to create two things:
- The dataset that will be embedded in the heroku app. This dataset will be called each time the user wants to modify something.
- The script that will drive the application (callbacks, layout and so on ...)

We will then finally send these two files onto a heroku repository and then the application will run and be available online.

#### Import the stock auction dataset

In [ ]:
import yfinance as yf

# Get information about FTSE companies 
FTSE_companies = pd.read_excel("Brexit_datas/stock_actions/FTSE_100_list.xlsx")
tickers_FTSE = list(FTSE_companies.Ticker)
FTSE_companies.set_index("Ticker",inplace=True)

# Get the data for the FTSE companies
stock_action_FTSE = yf.download(tickers_FTSE,'2015-01-01','2020-08-01',show_errors=False)['Adj Close']

# Clean corrupted columns
stock_action_FTSE = stock_action_FTSE.dropna(axis=1).reset_index()

#### Generate the dataset that will be embeded into the heroku application

In [ ]:
# Set the windows over which we will compute the mooving average
windows_avg_quotations = 9
windows_avg_stock = 9

# Generate a date index to remove sparsity from the datas
date_range = pd.DataFrame(pd.date_range(start=stock_action_FTSE.Date.dt.date.min(),end=stock_action_FTSE.Date.dt.date.max()))
date_range.columns = ["Date"]
# Merge the data range
stock_action_avg = pd.merge(stock_action_FTSE,date_range,on="Date",how="right")
cols_without_date = list(stock_action_avg.columns)
cols_without_date.remove("Date")
# Perform a linear interpolation for missing values 
stock_action_avg.loc[:,cols_without_date] = stock_action_avg.loc[:,cols_without_date].interpolate(axis=0,method="linear",limit_direction="forward")
# Mooving average operation
stock_action_avg.loc[:,cols_without_date] = stock_action_avg.loc[:,cols_without_date].rolling(windows_avg_stock,center=True).mean()
stock_action_diff = stock_action_avg.copy()
# Compute the derivative
stock_action_diff.loc[:,cols_without_date] = stock_action_diff.loc[:,cols_without_date].diff(axis=0)
# Get ride of the head and tail values
stock_action_diff = stock_action_diff[~stock_action_diff.iloc[:,1].isna()]
# Take the absolute value of the derivative
stock_action_diff.loc[:,cols_without_date] = stock_action_diff.loc[:,cols_without_date].abs()
# Set the Date as an index 
stock_action_diff.set_index("Date",inplace=True)
stock_action_avg.set_index("Date",inplace=True)

# Generate a date index to remove sparsity from the datas
date_range = pd.date_range(start=quotebank_brexit.date.dt.date.min(),end=quotebank_brexit.date.dt.date.max())
quotes_count = pd.DataFrame(date_range,columns=["date"])
# Count the number of quotations per date 
count_quotebank = quotebank_brexit.date.dt.date.to_frame().groupby("date").size().reset_index(name="Count")
count_quotebank["date"] = pd.to_datetime(count_quotebank["date"])
# Merge with the data range
quotes_count = pd.merge(quotes_count,count_quotebank,on="date",how="left")
# Fill date without quotations
quotes_count.loc[quotes_count["Count"].isna(),"Count"] = 0
# Compute the cumulative sum
quotes_count["Count"] = quotes_count["Count"].cumsum()
# Compute the mooving average 
quotes_count["moo_avg"] = quotes_count["Count"].rolling(windows_avg_quotations,center=True).mean()
# Let keep only quotes after 2016 (not enough data before)
quotes_count = quotes_count[quotes_count.date.dt.year >= 2016]
# Compute the differential
quotes_count["differential"] = quotes_count.moo_avg.diff()
# Get ride of the head and tail values
quotes_count = quotes_count[~quotes_count["differential"].isna()]
quotes_count.set_index("date",inplace=True)

# Reindex 
stock_action_diff = stock_action_diff.loc[quotes_count.index]
stock_action_avg = stock_action_avg.loc[quotes_count.index]

# Let compute the pearson correlation coefficient
pearson_results = []
# Significance level
alpha = 0.05

# For each stock auction
for col in cols_without_date:
    # Compute pearson correlation coeff and associated p-value
    pearson, p_value = stats.pearsonr(quotes_count.differential,stock_action_diff.loc[:,col])
    pearson_results.append([pearson, p_value, col])

# Convert to a DataFrame
pearson_results = pd.DataFrame(pearson_results,columns = ["pearson_value","p-value","action"])
# Keep stock auctions for which the correlation is significant 
pearson_results = pearson_results[pearson_results["p-value"] < alpha]
# Compute absolute value of pearson coeff
pearson_results["pearson_value_abs"] = pearson_results["pearson_value"].abs()
pearson_results.set_index("action",inplace=True)
# Keep only stock_auction for which the absolute value of r is superior to 0.15
pearson_results = pearson_results[pearson_results.pearson_value.abs() > 0.15]
pearson_results = pd.merge(pearson_results,FTSE_companies[["Company Name"]],left_index=True,right_index=True,how="left")
pearson_results = pearson_results.sort_values(by="pearson_value_abs",ascending=False)

# Export results 
if not exists("Dash_datas/dash_datas_stock_pearson.csv"):
    pearson_results.to_csv("Dash_datas/dash_datas_stock_pearson.csv")

print(pearson_results.head())

action_retained = list(pearson_results.index)

# Keep only some stock auctions 
stock_action_diff = stock_action_diff.loc[:,action_retained]
dash_datas_action = pd.merge(quotes_count.drop(columns=["Count","moo_avg"]),stock_action_diff,left_index=True,right_index=True)
dash_datas_action.rename(columns={"differential":"quotes_differential"},inplace=True)
# Export results 
if not exists("Dash_datas/dash_datas_stock_auction.csv.gz"):
    dash_datas_action.to_csv("Dash_datas/dash_datas_stock_auction.csv.gz")

dash_datas_action.head()

#### Python script embedded into the heroku app

In [ ]:
import dash
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import statsmodels

# Initialize the app
app = dash.Dash(__name__)
server = app.server

# Import the datas 
dash_datas_action = pd.read_csv("Dash_datas/dash_datas_stock_auction.csv.gz",compression="infer")
dash_datas_action.set_index("date",inplace=True)
dash_pearson = pd.read_csv("Dash_datas/dash_datas_stock_pearson.csv")
dash_pearson.set_index("action",inplace=True)

# Set the options 
options = [{"label": action, "value": action,
            "style" : {"color":"black","font-family":"Helvetica","font-size": 18}} 
            for action in list(dash_pearson.index)]

# Define the layout of the application
app.layout = html.Div([
                        dcc.Dropdown(id='drop-down',
                                    style={"color":"black",
                                           "font-family":"Helvetica",
                                           "margin-left":"15%",
                                           "width":"70%",
                                           "margin-right":"15%",
                                           "font-size": 18},
                                    options=options,
                                    searchable=False,
                                    value=dash_pearson.index.values[0]),
                        dcc.Graph(id="dashboard",responsive=True)
    ]
)

# Update the dashboard 
@app.callback(Output("dashboard", "figure"),
              Input("drop-down","value"))
def update_action_dashboard(action):
      # Style the dashboard 
      specs=[[{}, {"rowspan":2}],[{}, None]]
      # Initialize the figure 
      fig = make_subplots(rows=2,cols=2,shared_xaxes=True,vertical_spacing=0.25,specs=specs,column_widths=[0.6, 0.4],
                        subplot_titles=("","Pearson coefficient r = " + str(round(dash_pearson.loc[action,"pearson_value"],3)),""))
      # Derivative plot
      fig.add_trace(go.Scatter(x=dash_datas_action.index,y=dash_datas_action.quotes_differential,
                              mode="lines",name="Derivative [quotations]"),row=1,col=1)
      fig.add_trace(go.Scatter(x=dash_datas_action.index,y=dash_datas_action.loc[:,action],
                              mode="lines",name="Derivate's absolute value [" + action + "]"),row=2,col=1)
      # Scatter plots 
      regline_fig = px.scatter(dash_datas_action, x="quotes_differential", y=action, trendline="ols",
                              trendline_color_override="black",color_discrete_sequence=["#ff8812"])
      
      # Transfer trace to the figure 
      for trace in regline_fig["data"]:
            fig.add_trace(trace,row=1,col=2)

      # Tune the figure 
      fig.update_layout(legend=dict(
                        font={"family":"Helvetica"},
                        orientation="h",
                        yanchor="top",
                        y=1.2,
                        xanchor="left",
                        x=0),
                        )
      fig.update_layout(title = {'text': "Correlation between brexit agitation and '" + dash_pearson.loc[action,"Company Name"].lower() + "' auction",
                        'y':0.02,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'bottom',"font":{"family":"Helvetica","size":22}})
      fig.update_annotations(font_family='Helvetica')
      fig['layout']['xaxis']['rangeslider']['visible']=True
      fig['layout']['xaxis3']['title']='Date'
      fig['layout']['xaxis3']['title']['font']['family'] = 'Helvetica'
      fig['layout']['xaxis2']['title']='Derivative [quotations]'
      fig['layout']['yaxis2']['title']='Derivative [' + action + ']'
      fig['layout']['yaxis2']['side']= 'right'
      fig['layout']['yaxis2']['title']['font']['family'] = 'Helvetica'
      fig['layout']['xaxis2']['title']['font']['family'] = 'Helvetica'
      fig['layout']['titlefont']['family'] = 'Helvetica'

      fig.update_layout(height=1200)
      return fig

if __name__ == '__main__':
    app.run_server()